In [ ]:
# default_exp atmos

# Atmospheric Correction

> summary

This module contains a 6SV1.1 Atmospheric Correction class that computes the pixel radiance given some parameters and a servable interactive SNR widget. 6SV is an open souce radiative transfer code that predicts the solar spectrum received by a sensor facing the Earth. Since it is written in Fortran, we use a Python wrapper called Py6S to expose the functionality and that means we are limited to using 6SV1.1 rather than the newer 6SV2.1. 

In [ ]:
#hide

# documentation extraction for class methods
from nbdev.showdoc import *

# unit tests using test_eq(...)
from fastcore.test import *

# monkey patching class methods using @patch
from fastcore.foundation import *
from fastcore.foundation import patch

# imitation of Julia's multiple dispatch using @typedispatch
from fastcore.dispatch import typedispatch

# bring forth **kwargs from an inherited class for documentation
from fastcore.meta import delegates

In [ ]:
#export

from fastcore.foundation import patch
from fastcore.meta import delegates
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime as dt
import os

import param
import panel as pn
pn.extension()

import holoviews as hv
hv.extension('bokeh')

from Py6S import *


In [ ]:
#export

#from openhsi.data import *




## Atmospheric Profile from Balloon Sounding


Find the closest station at http://weather.uwyo.edu/upperair/sounding.html
to use the atmospheric sounding on the day.
find the station number and region code (for example, south pacific is `pac` and new zealand is `nz`)
Default is Willis Island in Queensland, Australia.
https://py6s.readthedocs.io/en/latest/helpers.html#importing-atmospheric-profiles-from-radiosonde-data

solar zenith angle already taken into account

## Aerosol Profile from Aeronet

Permanently broken. 


In [ ]:
#export

class Model6SV():
    
    def __init__(self, lat:"degrees" = -17.7, lon:"degrees" = 146.1, # Queensland
                 z_time:"zulu" = dt.strptime("2021-05-26 03:26","%Y-%m-%d %H:%M"),
                 station_num:int = 94299, region:str = "pac",
                 alt:"km" = 0.12, zen:"degrees" = 0., azi:"degrees" = 0., 
                 tile_type:GroundReflectance = GroundReflectance.GreenVegetation,
                 aero_profile:AeroProfile = AeroProfile.Maritime,
                 λ_array:"array [μm]" = np.arange(0.4, .8, 0.004),
                 sixs_path="assets/sixsV1.1"):
        
        self.λ_array = λ_array
        
        s = SixS(sixs_path)
        
        #Atmosphere
        s.atmos_profile = AtmosProfile.PredefinedType(AtmosProfile.MidlatitudeSummer)
        
        # crude calculation if daytime is 12Z or 0Z based on time and longitude
        z_hour = 0 if ((z_time.hour + int(lon/30))%24 - 12)/12. < 0.5 else 12
        radiosonde_url = f"http://weather.uwyo.edu/cgi-bin/sounding?region={region}&TYPE=TEXT%3ALIST&YEAR={z_time.year}&MONTH={z_time.month:02d}&FROM={z_time.day:02d}{z_hour:02d}{z_time.minute:02d}&TO={z_time.day:02d}{z_hour:02d}&STNM={station_num}"
        #print(radiosonde_url)
        s.atmos_profile = SixSHelpers.Radiosonde.import_uow_radiosonde_data(radiosonde_url,AtmosProfile.MidlatitudeSummer)
        
        # website at http://weather.uwyo.edu/cgi-bin/sounding?region=pac&TYPE=TEXT%3ALIST&YEAR=2021&MONTH=05&FROM=1212&TO=1212&STNM=94299
        
        # this can be custom?
        s.aero_profile = AeroProfile.PredefinedType(aero_profile)
        #s = SixSHelpers.Aeronet.import_aeronet_data_fixed(SixSHelpers.Aeronet,s,"assets/20190101_20211231_Lucinda.ONEILL_lev15","26-05-2021 03:26")
        s.visibility = 40 # km
        
        #Viewing and sun geometry
        s.geometry = Geometry.User()
        #s.geometry.solar_z = 0; s.geometry.solar_a = 0; s.geometry.view_z = 0; s.geometry.view_a = 0
        s.geometry.day = z_time.day
        s.geometry.month = z_time.month
        s.geometry.from_time_and_location(lat, lon, f"{z_time.year}-{z_time.month:02d}-{z_time.day:02d} {z_time.hour}:{z_time.minute}:{z_time.second}", zen, azi)
        
        #Altitude
        s.altitudes = Altitudes()
        s.altitudes.set_sensor_custom_altitude(alt) # km
        s.altitudes.set_target_sea_level()
        if tile_type is not None: s.ground_reflectance = GroundReflectance.HomogeneousLambertian(tile_type) 

        self.s = s
        
        self.__call__()
        
    def rad2photons(self):
        self.photons = self.radiance/( 1.98644582e-25/(self.λ*1e-6) )
        
    def __call__(self) -> None:
        self.λ, self.radiance = SixSHelpers.Wavelengths.run_wavelengths(self.s,self.λ_array,output_name="pixel_radiance")
        print("6SV computation done")
        
        df = pd.DataFrame({"wavelength":self.λ,"radiance":self.radiance})
        df.set_index("wavelength",inplace=True)
        df.interpolate(method="cubicspline",axis="index",limit_direction="both",inplace=True)
        self.radiance = df["radiance"].to_numpy()
        self.rad2photons()
    
    def show(self):
        plt.plot(self.λ*1000,self.radiance,label="computed radiance")
        plt.xlabel("wavelength (nm)")
        plt.ylabel("radiance (W/m$^2$/sr/$\mu$m)")
        plt.legend()
        plt.minorticks_on()

